# Using Environ for Solvation Effects on Isolated Systems

## Tutorial Setup

Import all the Python modules needed in this tutorial. Most of these are very common in scientific computing, some are popular tools in atomistic simulations.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.io import read, write
from ase.build import molecule
from ase.visualize import view
from ase.calculators.espresso import Espresso

Setting up environment variables that are needed in order to submit simulations using Quantum Espresso through ASE

In [ ]:
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

Some basic constants that may be useful later in the tutorials

In [ ]:
eV2Ry = 13.605662285137 # energy conversion factor
eV2kcal_mol = 23.0609 # energy conversion factor
bohr2ang = 0.5291772 # length conversion factor
ang2bohr = 1./bohr2ang

## Local Functions and Classes Used for the Tutorial

Basic functions to extract data from pw.x output files

In [ ]:
def get_total_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the total energies from each scf calculation

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('!'):
            energies.append(float(line.split()[4]))
    return energies

def get_scf_energy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the energies from each scf step

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    energies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('total energy'):
            energies.append(float(line.split()[3]))
    return energies

def get_scf_accuracy(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the energies from each scf step

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        energies = list of floats
    """
    accuracies=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('estimated scf accuracy'):
            accuracies.append(float(line.split()[4]))
    return accuracies

def get_total_force(filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, 
    extract the total force acting on the atoms from each scf calculation

    Input Variables:
        filename = name of pw.x output file
    
    Output Variables:
        forces = list of floats
    """
    forces=[]
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('Total force'):
            forces.append(float(line.split()[3]))
    return forces


def get_bond_length(i,j,filename='espresso.pwo'):
    """
    Given a filename corresponding to a pw.x output, and the indexes of two
    atoms in the simulation (using Python notation, starting from 0)
    extract the bond length for each scf step in the simulation

    Input Variables:
        i = index of first atom
        j = index of second atom
        filename = name of pw.x output file
    
    Output Variables:
        bonds = list of floats
    """
    bonds=[]
    ri = np.zeros(3)
    rj = np.zeros(3)
    lines = [line for line in open(filename, 'r')]
    for line in lines:
        if line.strip().startswith('number of atoms'):
            nat = int(line.split()[4])
            break
    if i < 0 or i >= nat : 
        print('Error, index i must be >= 0 and < nat')
        return
    if j < 0 or j >= nat : 
        print('Error, index j must be >= 0 and < nat')
        return
    if i == j : 
        print('Error, indexes i and j must be different')
        return
    iat = -1
    for line in lines:
        if iat >= 0 and iat < nat : 
            if iat == i : ri = np.array([line.split()[1:4]],dtype=float)
            if iat == j : rj = np.array([line.split()[1:4]],dtype=float)
            iat += 1 
        if iat == nat : 
            bonds.append(np.sqrt(np.sum((ri-rj)**2)))
            iat = -1
        if line.strip().startswith('ATOMIC_POSITIONS'):
            iat = 0
    return bonds

Cubefile visualization tools

In [ ]:
class cubefile():
    """
    Class to read and manipulate cubefiles
    """

    def __init__(this,filename,delta=1.e-5):
        lines = [line for line in open(filename, 'r')]
        this.nat = int(lines[2].split()[0])
        this.origin = np.array(lines[2].split()[1:4],dtype=float)
        this.n = np.zeros(3,dtype=int)
        this.dgrid = np.zeros(3)
        for i in range(3):
            this.n[i] = int(lines[3+i].split()[0])
            this.dgrid[i] = float(lines[3+i].split()[i+1])
        this.atmnum = np.zeros(this.nat,dtype=int)
        this.atmchg = np.zeros(this.nat)
        this.atmpos = np.zeros((3,this.nat))
        for i in range(this.nat):
            this.atmnum[i] = int(lines[6+i].split()[0])
            this.atmchg[i] = float(lines[6+i].split()[1])
            this.atmpos[:,i] = np.array(lines[6+i].split()[2:5],dtype=float)
        this.ntot = this.n[0]*this.n[1]*this.n[2]
        this.data = np.zeros(this.ntot) 
        i = 0
        for line in lines[6+this.nat:-1] : 
            this.data[i:i+6]=[ float(s) for s in line.split()]
            i += 6
        this.data[i:]=[ float(s) for s in lines[-1].split()]
        this.data = this.data.reshape((this.n[0],this.n[1],this.n[2]))
        this.r=np.zeros((3,this.n[0],this.n[1],this.n[2]))
        this.x=np.arange(0.,this.dgrid[0]*this.n[0]-delta,this.dgrid[0])+this.origin[0]
        this.y=np.arange(0.,this.dgrid[1]*this.n[1]-delta,this.dgrid[1])+this.origin[1]
        this.z=np.arange(0.,this.dgrid[2]*this.n[2]-delta,this.dgrid[2])+this.origin[2]
        this.r[0],this.r[1],this.r[2]=np.meshgrid(this.x,this.y,this.z,indexing='ij')

    def to_line(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            axis = this.r[0,:,icenter[1],icenter[2]]
            value = this.data[:,icenter[1],icenter[2]]
        elif axis == 1 :
            axis = this.r[1,icenter[0],:,icenter[2]]
            value = this.data[icenter[0],:,icenter[2]]
        elif axis == 2 :
            axis = this.r[2,icenter[0],icenter[1],:]
            value = this.data[icenter[0],icenter[1],:]
        return axis,value
    
    def to_line_planar_average(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            axis = this.r[0,:,icenter[1],icenter[2]]
            value = np.mean(this.data,(1,2))
        elif axis == 1 :
            axis = this.r[1,icenter[0],:,icenter[2]]
            value = np.mean(this.data,(0,2))
        elif axis == 2 :
            axis = this.r[2,icenter[0],icenter[1],:]
            value = np.mean(this.data,(0,1))
        return axis,value
    
    def to_surface(this,center,axis):
        icenter = np.rint(center/this.dgrid).astype('int')
        icenter = icenter - this.n * np.trunc(icenter//this.n).astype('int') 
        if axis == 0 :
            xx = this.r[1,icenter[0],:,:]
            yy = this.r[2,icenter[0],:,:]
            zz = this.data[icenter[0],:,:]
        elif axis == 1 :
            xx = this.r[0,:,icenter[1],:]
            yy = this.r[2,:,icenter[1],:]
            zz = this.data[:,icenter[1],:]
        elif axis == 2 :
            xx = this.r[0,:,:,icenter[2]]
            yy = this.r[1,:,:,icenter[2]]
            zz = this.data[:,:,icenter[2]]
        return xx,yy,zz

## Modeling a Water Molecule in Solution

We can use the ase.build.molecule() method to create a water molecule in vacuum. We can also use ASE to visualize it: checking every step of your simulation is a great way to avoid troubles, learning to visualize your data is a key skill for debugging.

In [ ]:
atoms = molecule('H2O')
view(atoms, viewer="x3d")

ASE has created an isolated molecule. However, QE is a tool to simulate periodic systems, so we need to put our molecule in a periodic simulation cell. For simplicity, but also to ensure a better behaviour of the electrostatics interactions, a cubic cell is the best choice to approximate isolated systems. 

In [ ]:
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

QE is a plane-wave code that uses pseudopotentials to approximate core electrons, so that the resulting electron density is smoother and we can use a smaller number of plane-waves. Make sure the pseudopotentials are downloaded in the right folder (specified in the psedo_dir keyword). NOTE: in QE the choice of pseudopotential determines the DFT functionals that you will be using. 

For this tutorial, we are using the Perdew–Burke-Ernzerhof (PBE) functional and we are using Ultrasoft (us) PP for all the atoms. The pseudopotentials are downloaded from the Quantum Espresso website (e.g. from https://www.quantum-espresso.org/wp-content/uploads/upf_files/O.pbe-rrkjus.UPF)

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

### Running a Single SCF Calculation with QE

The following cell specifies the input keywords for a QE calculation with the pw.x code. You can find all of the available keywords and their default values in QE documentation and online, e.g., at https://www.quantum-espresso.org/Doc/INPUT_PW.html.

We will start with a single SCF calculation. NOTE: at this time we have not specified any environment yet, this is just a QE calculation for an isolated water molecule in a periodic cubic cell.

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

The following cell will use ASE to run pw.x on the system with the keywords specified above.

In [ ]:
energy = atoms.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

In [ ]:
plt.subplot(2,1,1)
scf_energies = get_scf_energy('espresso.pwo')
plt.plot(scf_energies,'o-')
plt.ylabel('total energy (Ry)')
plt.subplot(2,1,2)
scf_accuracies = get_scf_accuracy('espresso.pwo')
plt.semilogy(scf_accuracies,'o-')
plt.ylabel('estimated SCF acc. (Ry)')
plt.show()

### Using Environ for PBC Corrections in 0D

We will now use Environ for one of the least invasive environment description, i.e., to remove PBC artifacts from our periodic cell calculation. NOTE that there are many alternative ways of removing PBC artifacts already implemented in QE from an isolated system in vacuum. This is mostly to test that Environ works and to introduce the topic of PBC corrections. 

In order to run a pw.x calculation with Environ, we need to specify it in the command line using the --environ keyword after the executable name.

In [ ]:
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x --environ -in PREFIX.pwi > PREFIX.pwo"

The following cell uses bash to generate a new file in the working folder. This file, named environ.in, is the input used by Environ to setup the environment conditions.

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 0.1 ! this controls when Environ kicks in
   environ_type = 'vacuum' ! this is the default value, no environment effects
   env_electrostatic = .true. ! this will still activate Environ for electrostatics
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
/
EOF

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
energy = atoms.get_potential_energy()
print(f"Energy in vacuum + parabolic correction = {energy:.3f} eV")

### Debugging and Visualization with Environ

We can restart a pw.x calculation and restart Environ with it. This allows to run a short simulation on a converged system (thus, only 1 SCF step) with a higher verbosity in Environ. When verbose = 1, Environ will produce a debug file named environ.debug with some useful information.

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 2
   environ_thr = 0.1
   environ_type = 'vacuum'
   environ_restart = .true.
   env_electrostatic = .true.
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
/
EOF

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'restart',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
energy = atoms.get_potential_energy()
print(f"Energy in vacuum + parabolic correction = {energy:.3f} eV")

Running Environ with a verbosity of 2 will generate some (pretty sizable) files in the Gaussian cubefile format. These files contain information on the simulation cell, the grid, the position and charges of the atoms, and the value of a scalar field on the gridpoints of the simulation cell. For our specific simulation, the three cubefiles that Environ prints are called
* dvtot : this contains the correction to the Kohn-Sham potential that Environ passes back to QE
* vreference: this is the electrostatic potential as computed for a periodic system using FFTs. This equivalent to the potential computed by QE, but with the minor difference that atoms are treated as smeared charges (Gaussians, default spread of 0.5 a.u.) by Environ (instead of point charges and pseudo-potentials)
* velectrostatic: this is the electrostatic potential in the presence of the environment effects. Also in this case, atoms are modelled as Gaussians.

In [ ]:
vref=cubefile('vreference.cube')
vel=cubefile('velectrostatic.cube')
dv=cubefile('dvtot.cube')

In [ ]:
x,y=vref.to_line(atoms.get_center_of_mass()*ang2bohr,2)
plt.plot(x,y)
x,y=vel.to_line(atoms.get_center_of_mass()*ang2bohr,2)
plt.plot(x,y)
x,y=dv.to_line(atoms.get_center_of_mass()*ang2bohr,2)
plt.plot(x,y)
plt.ylim(-0.05,0.05)
plt.show()

### Relax in Vacuum

The solvation free energy in a continuum medium framework is defined as the difference in energy between a relaxed structure in vacuum and a relaxed structure in the continuum medium. Here we will perform the first step of the calculation.

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'restart',
        'pseudo_dir': '../pseudos',
        'calculation': 'relax',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 0.1
   environ_type = 'vacuum'
   environ_restart = .true.
   env_electrostatic = .true.
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
/
EOF

In [ ]:
energy_vacuum = atoms.get_potential_energy()
print(f"Relaxed total energy in vacuum = {energy_vacuum:.3f} eV")

The following cell will copy the output of the simulation for future use.

In [ ]:
! cp espresso.pwo H2O_vacuum.out

In [ ]:
energies=get_total_energy()
forces=get_total_force()
bonds=get_bond_length(0,1)

In [ ]:
plt.figure(figsize=(6,8))
plt.subplot(3,1,1)
plt.plot(np.arange(1,len(energies)+1),energies,'o-')
plt.ylabel('Total Energy (Ry)')
plt.subplot(3,1,2)
plt.plot(np.arange(1,len(bonds)+1),bonds,'o-')
plt.ylabel('O-H Bond Length (a.u.)')
plt.subplot(3,1,3)
plt.semilogy(np.arange(1,len(forces)+1),forces,'o-')
plt.ylabel('Total Force (Ry/a.u.)')
plt.show()

### SCF in Solution: Problems and Solutions (Pun Intended)

Before running a relax calculation in solution, let us examine the main parameters of Environ for simulating a system in water. To show some of the typical problems with using a continuum solvation model, we will perform a short SCF simulation with low accuracy (faster). 

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_water'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-6, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 0.01
   environ_type = 'water'
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-2
   !
/
EOF

In [ ]:
energy = atoms.get_potential_energy()

In [ ]:
! cp espresso.pwo H2O_water_bad.out

We can visualize the properties of the system along the simulation. What problems do you see?

In [ ]:
plt.subplot(2,1,1)
scf_energies = get_scf_energy('H2O_water_bad.out')
plt.plot(scf_energies,'o-')
plt.ylabel('total energy (Ry)')
plt.subplot(2,1,2)
scf_accuracies = get_scf_accuracy('H2O_water_bad.out')
plt.semilogy(scf_accuracies,'o-')
plt.ylabel('estimated SCF acc. (Ry)')
plt.show()

We can now re-run our calculation with better parameters for Environ. The goal is to choose parameters that minimize the number of SCF steps required to converge.

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_water'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-4, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 1.
   environ_type = 'water'
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-10
   !
/
EOF

In [ ]:
energy = atoms.get_potential_energy()

In [ ]:
plt.subplot(2,1,1)
scf_energies = get_scf_energy('H2O_water_bad.out')
plt.plot(scf_energies,'o-')
scf_energies = get_scf_energy('espresso.pwo')
plt.plot(scf_energies,'o-')
plt.ylabel('total energy (Ry)')
plt.subplot(2,1,2)
scf_accuracies = get_scf_accuracy('H2O_water_bad.out')
plt.semilogy(scf_accuracies,'o-')
scf_accuracies = get_scf_accuracy('espresso.pwo')
plt.semilogy(scf_accuracies,'o-')
plt.ylabel('estimated SCF acc. (Ry)')
plt.show()

### Relax in Solution

The calculation is a standard geometry relaxation calculation, but in the presence of a continuum dielectric with the shape defined
by the electronic density, according to the  model of Fattebert and Gygi, Int J Quantum Chem 93, 139 (2003), revised by O. Andreussi, I. Dabo and N. Marzari, J. Chem. Phys. 136, 064102 (2012) 

Mind that the parabolic correction (or point-countercharge method) is used for both the calculation in vacuum and in the solvent.

Additional terms of solvation can be computed by changing the specific keywords:
* env_surface_tension: surface tension of the solvent. If different from zero, a cavitation energy is computed according to a revised version of the formula in Scherlis et al. JCP 124, 074103, 2006 and Andreussi et al. J. Chem. Phys. 2012. 
* env_pressure: external pressure of the medium. If different from zero, a PV energy term is added to the total energy, according to the method in Cococcioni et al. PRL 94, 145501, 2005.

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'relax',
        'prefix': 'H2O_water'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 1
   environ_type = 'water'
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-11
   !
/
EOF

In [ ]:
energy_solution = atoms.get_potential_energy()
print(f"Relaxed total energy in solution = {energy_solution:.3f} eV")

In [ ]:
! cp espresso.pwo H2O_water.out

In [ ]:
energies=get_total_energy()
forces=get_total_force()
bonds=get_bond_length(0,1)

In [ ]:
plt.figure(figsize=(6,8))
plt.subplot(3,1,1)
plt.plot(np.arange(1,len(energies)+1),energies,'o-')
plt.ylabel('Total Energy (Ry)')
plt.subplot(3,1,2)
plt.plot(np.arange(1,len(bonds)+1),bonds,'o-')
plt.ylabel('O-H Bond Length (a.u.)')
plt.subplot(3,1,3)
plt.semilogy(np.arange(1,len(forces)+1),forces,'o-')
plt.ylabel('Total Force (Ry/a.u.)')
plt.show()

In [ ]:
deltaG = ( energy_solution - energy_vacuum ) * eV2kcal_mol
print('The computed solvation free energy of a water molecule is = {:4.1f} kcal/mol'.format(deltaG))

### Digging in the Environ Parameters and Keywords

The input files of Environ that we have used so far contained very few keywords, as the `environ_type = 'water'` keyword takes care of setting most of the parameters automatically. In practice, the following would have been the manual way of specifying the same type of calculation. 

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 1
   env_static_permittivity = 78.3
   env_pressure = -0.36
   env_surface_tension = 47.9
   !
/
&BOUNDARY
   ! 
   solvent_mode = 'electronic'
   rhomax = 0.005
   rhomin = 0.0001
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   solver = 'cg'
   mix = 0.6
   tol = 1.d-11
   !
/
EOF

A possibility, in the following, could be to explore an alternative definition of the boundary between the environment and the system. Specifying a value of `solvent_mode = 'ionic'` allows to define a continuum region constructed using interlocking spheres centered on the nuclei ("soft-spheres") as described in G. Fisicaro, L. Genovese, O. Andreussi, S. Mandal, N.N. Nair, N. Marzari, and S.Goedecker, J. Chem. Theor. Comput. 13, 3829 (2017). Note that this different cavity relies on different parameters (including van der Waals radii for the different elements) and on a separate parametrization of the corresponding solvation model. The keyword `environ_type = 'water'` automatically sets these parameters for the SSCS (soft-sphere continuum solvation) model of Fisicaro et al.

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 1
   env_type = 'water'
   !
/
&BOUNDARY
   ! 
   solvent_mode = 'ionic'
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-11
   !
/
EOF

## Solvent-Aware Interface

This example shows how to use pw.x to calculate the energy of a water cluster (thanks to Nicolas Hörmann for the structure) using a solvent-aware interface, as described in O. Andreussi, N.G. Hörmann, F. Nattino, G. Fisicaro, S. Goedecker, and N. Marzari, J. Chem. Theory Comput. 15, 1996 (2019).

This feature is particularly useful to model branched/open/porous systems, where pockets that are too small to fit a solvent molecule  should remain dielectric-free. 

Two calculations are carried out. The first calculation includes a standard interface between the quantum mechanical and the continuum region, such that the dielectric unphysically fills the small pocket in between the  water molecules. The second calculation models instead a solvent-aware interface, which prevents the dielectric continuum to enter the central 'hole' of the cluster.

In [ ]:
atoms = read('H2O_cluster.xyz')

In [ ]:
atoms.set_cell(14. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

In [ ]:
view(atoms, viewer="x3d")

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_cluster'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 1
   environ_thr = 1
   environ_type = 'water'
   !
/
&BOUNDARY
   !
   solvent_mode = 'electronic'
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-11
   !
/
EOF

In [ ]:
energy_no_sa = atoms.get_potential_energy()
print('Total energy of the water cluster without solvent-aware = {:10.3f} eV'.format(energy_no_sa))

In [ ]:
! cp espresso.pwo H2O_cluster_no.out

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 1
   environ_thr = 1
   environ_type = 'water'
   !
/
&BOUNDARY
   !
   solvent_mode = 'electronic'
   filling_threshold = 0.85
   solvent_radius = 2.6 
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-11
   !
/
EOF

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_cluster'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
energy_sa = atoms.get_potential_energy()
print('Total energy of the water cluster with solvent-aware = {:10.3f} eV'.format(energy_sa))

In [ ]:
! cp espresso.pwo H2O_cluster_sa.out

In [ ]:
print('The difference due to solvent-aware is = {:4.3f}'.format(energy_sa-energy_no_sa))

## Running Environ as a Post-Processing Tool

This last example shows how to use Environ as a stand-alone program. You will need to compile the Environ/programs by typing `make` inside that folder. The compilation should generate a `driver` executable that can perform an environ calculation on a 'frozen' system (atoms and electronic density) specified in a cubefile. 

For the purpose of this example, we can use the `system.cube` file included in this folder. However, this type of file can be generated relatively easily from a pw.x calculation with Environ, by setting the verbosity to 4 or higher and manually hacking the `electrons.cube` file generated as a result. The following three command cells and some manual editing (detailed in the following) can be skipped.

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 4
   environ_thr = 1
   environ_type = 'water'
   environ_restart = .true.
   !
/
&BOUNDARY
   !
   solvent_mode = 'electronic'
   filling_threshold = 0.85
   solvent_radius = 2.6 
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-11
   !
/
EOF

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'restart',
        'pseudo_dir': '../pseudos',
        'calculation': 'scf',
        'prefix': 'H2O_cluster'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
energy = atoms.get_potential_energy()

As a result of the above cells, we have now generated several cubefiles, including one that contains the optimized electronic density, named `electrons.cube`. This file contains part of the information needed by Environ to perform its calculations, but unfortunately its header misses the information on the atoms, which is instead saved in the `dvtot.cube` file. In order to generate the file that Environ needs we will perform the following steps:
1. Copy `electrons.cube` into our final file
2. Cut and paste the header section from `dvtot.cube`
3. Manually set the empty column near the atomic numbers with the charge of the ions (nuclei+core electrons)

Once we have a cubefile with all the relevant information, we can run the driver program of Environ to perform a frozen calculation, as follows:

In [ ]:
! /Users/oliviero/PWSCF/espresso-git/Environ/programs/driver -n from_cube -i environ.in -c system.cube

Let us save the `boundary_solvent.cube` file generated by this simulation, so that we can compare it wwith its non-solvent-aware counterpart.

In [ ]:
boundary_sa = cubefile('boundary_solvent.cube')

Now we can reset the `environ.in` file for a non-solvent-aware simulation (note that the driver can accept any filename for the environ input, allowing you to have multiple input files to use in the same folder)

In [ ]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 4
   environ_thr = 1
   environ_type = 'water'
   environ_restart = .true.
   !
/
&BOUNDARY
   !
   solvent_mode = 'electronic'
   !
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-11
   !
/
EOF

In [ ]:
! /Users/oliviero/PWSCF/espresso-git/Environ/programs/driver -n from_cube -i environ.in -c system.cube

In [ ]:
boundary_no = cubefile('boundary_solvent.cube')

We can now look at the boundary of the two calculations, with and without solvent-aware correction, along a line passing through the center of mass and directed along the x axis

In [ ]:
x,y=boundary_sa.to_line(atoms.get_center_of_mass()*ang2bohr,0)
plt.plot(x,y)
x,y=boundary_no.to_line(atoms.get_center_of_mass()*ang2bohr,0)
plt.plot(x,y)
plt.show()

or visualize a contourplot passing through the same point and perpendicular to the z axis

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
xx,yy,zz=boundary_no.to_surface(atoms.get_center_of_mass()*ang2bohr,2)
plt.axis('equal')
plt.contourf(xx,yy,zz)
plt.subplot(1,2,2)
xx,yy,zz=boundary_sa.to_surface(atoms.get_center_of_mass()*ang2bohr,2)
plt.axis('equal')
plt.contourf(xx,yy,zz)
plt.show()